In [1]:
# import libraries
import pandas as pd
import numpy as np
import random as rd
import subprocess
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
import openpyxl
import os
import seaborn as sns

# avoid warnings
pd.options.mode.chained_assignment = None

In [2]:
# load datasets
bk_1 = pd.read_csv('data/GM12878_gDNA_1.vcf', header=30,sep='\t', dtype='object')
bk_2 = pd.read_csv('data/GM12878_gDNA_2.vcf', header=30,sep='\t', dtype='object')
bk_3 = pd.read_csv('data/GM12878_gDNA_3.vcf', header=30,sep='\t', dtype='object')
bk_4 = pd.read_csv('data/GM12878_gDNA_4.vcf', header=30,sep='\t', dtype='object')
bk_5 = pd.read_csv('data/GM12878_gDNA_5.vcf', header=30,sep='\t', dtype='object')

sc_129 = pd.read_csv('data/GM12878_SC_129.vcf.gz', header=30,sep='\t', dtype='object')
sc_130 = pd.read_csv('data/GM12878_SC_130.vcf.gz', header=30,sep='\t', dtype='object')
sc_131 = pd.read_csv('data/GM12878_SC_131.vcf.gz', header=30,sep='\t', dtype='object')
sc_132 = pd.read_csv('data/GM12878_SC_132.vcf.gz', header=30,sep='\t', dtype='object')
sc_133 = pd.read_csv('data/GM12878_SC_133.vcf.gz', header=30,sep='\t', dtype='object')

# merge datasets
bk_merg = pd.concat([bk_1, bk_2.iloc[:,-1], bk_3.iloc[:,-1], bk_4.iloc[:,-1], bk_5.iloc[:,-1]], axis=1)
sc_merg = pd.concat([sc_129, sc_130.iloc[:,-1], sc_131.iloc[:,-1], sc_132.iloc[:,-1],  sc_133.iloc[:,-1]], axis=1)

In [3]:
sc_merg

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,201666260058_R01C01,201666260058_R01C02,201285550024_R06C01,201285550024_R05C01,201285550024_R04C01
0,1,752566,rs3094315,G,A,.,PASS,.,GT:GQ,1/1:2,1/1:1,1/1:9,1/1:9,1/1:9
1,1,753541,rs2073813,G,A,.,PASS,.,GT:GQ,./.:0,./.:0,0/0:7,0/0:7,0/0:7
2,1,770216,rs2905040,G,A,.,PASS,.,GT:GQ,0/0:3,0/0:3,0/0:3,0/0:3,0/0:3
3,1,776546,rs12124819,A,G,.,PASS,.,GT:GQ,0/0:2,./.:0,0/1:1,./.:0,0/1:3
4,1,781258,rs2980314,G,T,.,PASS,.,GT:GQ,0/0:3,0/0:3,0/0:3,0/0:3,0/0:3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294596,Y,28738253,rs2381338,A,G,.,PASS,.,GT:GQ,.:0,.:0,.:0,.:0,.:0
294597,Y,28760588,rs13303815,G,A,.,PASS,.,GT:GQ,.:0,0:2,.:0,.:0,0:1
294598,Y,28770162,cnvi37516747,G,"N,A",.,PASS,.,GT:GQ,.:0,.:0,.:0,.:0,.:0
294599,Y,28783112,cnvi37517006,T,"N,A",.,PASS,.,GT:GQ,.:0,.:0,.:0,.:0,.:0


In [11]:
# set the parameters

# set the paths
result_dir = '../server/results/windows/'
output_dir = '../server/outputs/windows/'
plot_dir = '../server/plots/windows/'

# chromosomes = bk_1.iloc[:,0].unique()[:-2] # X and Y excluded
chromosomes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
               '13', '14', '15', '16','17', '18', '19', '20', '21', '22']

all_positions = bk_merg.POS.to_list() # 294601 positions
column_names = bk_merg.columns.to_list()
bulk_ids = ['1','2','3','4','5']
singlecell_ids = ['129','130','131','132','133']
windows = ['w1','w2','w3','w4','w5']
perc = 20

In [ ]:
for chrom in chromosomes:
    bk_merg_chr = bk_merg[bk_merg['#CHROM'] == chrom]
    sc_merg_chr = sc_merg[sc_merg['#CHROM'] == chrom]

    bk_merg_chr.columns = column_names
    sc_merg_chr.columns = column_names

    bk_merg_chr['FORMAT'] = 'GT'
    sc_merg_chr['FORMAT'] = 'GT'

    bk_merg_chr.iloc[:,9:] = bk_merg_chr.iloc[:,9:].apply(lambda x : x.str.split(':').str.get(0))
    sc_merg_chr.iloc[:,9:] = sc_merg_chr.iloc[:,9:].apply(lambda x : x.str.split(':').str.get(0))

    bk_merg_chr.to_csv(os.path.join(output_dir, f'GM12878_gDNA_merged_chr{chrom}.vcf'), sep='\t', index=False)
    sc_merg_chr.to_csv(os.path.join(output_dir, f'GM12878_SC_merged_chr{chrom}.vcf.gz'), sep='\t', index=False)


In [5]:
#pd.read_csv('../server/outputs/GM12878_gDNA_merged_chr3.vcf',header=0,sep='\t', dtype='object').head(3)
pd.read_csv('../server/outputs/GM12878_SC_merged_chr3.vcf.gz',header=0,sep='\t', dtype='object').head(15)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,201666260058_R06C01,201666260058_R06C02,201662330194_R02C01,201662330194_R01C01,201662330194_R01C02
0,3,66894,rs9681213,G,A,.,PASS,.,GT,./.,0/1,./.,0/1,0/1
1,3,70895,rs9682794,T,C,.,PASS,.,GT,./.,./.,./.,./.,0/1
2,3,105793,rs1516332,C,T,.,PASS,.,GT,0/0,0/0,0/0,0/0,0/0
3,3,113534,rs9852028,G,T,.,PASS,.,GT,1/1,./.,0/1,0/1,0/1
4,3,114133,rs954824,T,C,.,PASS,.,GT,./.,0/1,0/1,0/1,0/1
5,3,120321,rs9860846,G,A,.,PASS,.,GT,1/1,1/1,1/1,1/1,1/1
6,3,125862,rs1400178,A,G,.,PASS,.,GT,0/0,0/1,./.,0/1,./.
7,3,131977,rs17048098,G,A,.,PASS,.,GT,0/1,./.,0/1,0/1,./.
8,3,145723,rs9790115,T,G,.,PASS,.,GT,1/1,./.,./.,0/1,0/1
9,3,149197,rs17259853,A,C,.,PASS,.,GT,0/0,0/0,0/0,./.,0/0


#### Filtering 1

In [15]:
for chrom in chromosomes:
    sc_merg = pd.read_csv(os.path.join(output_dir, f'GM12878_SC_merged_chr{chrom}.vcf.gz'), header=0, sep='\t', dtype='object')
    bk_merg = pd.read_csv(os.path.join(output_dir, f'GM12878_gDNA_merged_chr{chrom}.vcf'), header=0, sep='\t', dtype='object')

    bk_merg['FORMAT'] = 'GT'
    sc_merg['FORMAT'] = 'GT'

    print(bk_merg.shape[0])
    
    bk_merg.iloc[:,9:] = bk_merg.iloc[:,9:].apply(lambda x : x.str.split(':').str.get(0))
    bk_merg_calls = bk_merg[~(bk_merg.iloc[:, 9:] == './.').any(axis=1)] # bulk with only calls
    sc_merg_filt = sc_merg[sc_merg.POS.isin(bk_merg_calls.POS.to_list())==True] # single cell with the same bulk positions (calls bulk)
    
    print(bk_merg_calls.shape[0])
    print(sc_merg_filt.shape[0])
    print('-------')

    bk_merg_calls.to_csv(os.path.join(output_dir, f'GM12878_gDNA_merged_chr{chrom}_calls.vcf'), sep='\t', index=False)
    sc_merg_filt.to_csv(os.path.join(output_dir, f'GM12878_SC_merged_chr{chrom}_calls_bk.vcf.gz'), sep='\t', index=False)
    

22905
22421
22421
.-------.
23962
23452
23452
.-------.
17635
17256
17256
.-------.
14388
14071
14071
.-------.
17708
17318
17318
.-------.
16423
16064
16064
.-------.
15553
15179
15179
.-------.
15695
15311
15311
.-------.
12078
11800
11800
.-------.
13636
13369
13369
.-------.
14263
13937
13937
.-------.
14521
14253
14253
.-------.
8933
8727
8727
.-------.
9550
9339
9339
.-------.
9998
9738
9738
.-------.
9141
8897
8897
.-------.
9573
9344
9344
.-------.
7558
7400
7400
.-------.
6840
6638
6638
.-------.
7553
7403
7403
.-------.
4306
4220
4220
.-------.
4940
4787
4787
.-------.


In [27]:
import pandas as pd
from sklearn.metrics import jaccard_score

gen_dosage = {'0/0': 0, '0/1': 1, '1/0': 1, '1/1': 2, '1/2': 3, '2/2': 4, './.': 5}

j_list_pre = []
windows = []

for chrom in chromosomes:
    sc_merg = pd.read_csv(os.path.join(output_dir, f'GM12878_SC_merged_chr{chrom}_calls_bk.vcf.gz'), header=0, sep='\t', dtype='object')
    bk_merg = pd.read_csv(os.path.join(output_dir, f'GM12878_gDNA_merged_chr{chrom}_calls.vcf'), header=0, sep='\t', dtype='object')

    # divide the positions in 5 parts (20%)
    num_windows = 5
    window_size = int(len(bk_merg) / num_windows)

    start_index = 0

    dict_pre = []

    for window in range(num_windows):
        # Get the positions in the current window of the second file
        end_index = min(start_index + window_size, len(bk_merg))
        current_window_bk = bk_merg.iloc[start_index:end_index]

        # Add the positions in the current window of the second file to the list of windows
        windows.append(current_window_bk)

        # Remove the positions in the current window of the second file from the first file
        sc_merg_chr_wind_toimp = sc_merg[~sc_merg['POS'].isin(current_window_bk['POS'].tolist())]
        bk_merg = bk_merg[bk_merg['POS'].isin(current_window_bk['POS'].tolist())]
        sc_merg = sc_merg[sc_merg['POS'].isin(current_window_bk['POS'].tolist())]

        print(sc_merg.shape[0], bk_merg.shape[0], sc_merg_chr_wind_toimp.shape[0])

        vec_bulk = bk_merg[window].map(gen_dosage).tolist()
        vec_sing = sc_merg[window].map(gen_dosage).tolist()

        j_value = jaccard_score(vec_bulk, vec_sing, average='macro')
        j_list_pre.append(j_value)

        dict_js = {'chromosome': chrom, 'window': window, 'j_score': j_value}
        dict_pre.append(dict_js)

        sc_merg_chr_wind_toimp.to_csv(os.path.join(output_dir, f'GM12878_SC_merged_chr{chrom}_window{window}_toimp.vcf.gz'), sep='\t', index=False, compression='gzip')
        bk_merg.to_csv(os.path.join(output_dir, f'GM12878_gDNA_merged_chr{chrom}_window{window}_check.vcf'), sep='\t', index=False)


    pd.DataFrame(dict_pre).to_excel(os.path.join(result_dir, f'jaccard_scores_chr{chrom}_pre.xlsx'), index=False)


4484 4484 17937


KeyError: 0

In [74]:
wind1sc_toimp = pd.read_csv(f'../server/outputs/windows/GM12878_SC_merged_chr1_window1_toimp.vcf.gz', sep='\t')
wind1bk = pd.read_csv(f'../server/outputs/windows/GM12878_gDNA_merged_chr1_window1_check.vcf', sep='\t')
wind1sc = pd.read_csv(f'../server/outputs/windows/GM12878_SC_merged_chr1_window1_check.vcf.gz', sep='\t')


wind2sc_toimp = pd.read_csv(f'../server/outputs/windows/GM12878_SC_merged_chr1_window2_toimp.vcf.gz', sep='\t')
wind2bk = pd.read_csv(f'../server/outputs/windows/GM12878_gDNA_merged_chr1_window2_check.vcf', sep='\t')
wind2sc = pd.read_csv(f'../server/outputs/windows/GM12878_SC_merged_chr1_window2_check.vcf.gz', sep='\t')


wind3sc_toimp = pd.read_csv(f'../server/outputs/windows/GM12878_SC_merged_chr1_window3_toimp.vcf.gz', sep='\t')
wind3bk = pd.read_csv(f'../server/outputs/windows/GM12878_gDNA_merged_chr1_window3_check.vcf', sep='\t')
wind3sc = pd.read_csv(f'../server/outputs/windows/GM12878_SC_merged_chr1_window3_check.vcf.gz', sep='\t')

In [76]:
print(wind1bk.shape[0], wind1sc.shape[0], wind1sc_toimp.shape[0])
print(wind2bk.shape[0], wind2sc.shape[0], wind2sc_toimp.shape[0])
print(wind3bk.shape[0], wind3sc.shape[0], wind3sc_toimp.shape[0])

4581 4581 18324
4581 4581 18324
4581 4581 18324


In [82]:
wind1bk.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,201666260058_R06C01,201666260058_R06C02,201662330194_R02C01,201662330194_R01C01,201662330194_R01C02
0,1,33644352,rs680960,A,G,.,PASS,.,GT,1/1,1/1,1/1,1/1,1/1
1,1,33684101,rs1498319,C,T,.,PASS,.,GT,1/1,1/1,1/1,1/1,1/1
2,1,33690698,rs12045968,T,G,.,PASS,.,GT,0/0,0/0,0/0,0/0,0/0
3,1,33694810,rs7540951,C,T,.,PASS,.,GT,1/1,1/1,1/1,1/1,1/1
4,1,33700650,rs937429,T,C,.,PASS,.,GT,1/1,1/1,1/1,1/1,1/1


In [85]:
import pandas as pd
from sklearn.metrics import jaccard_score

samples = ['201666260058_R06C01', '201666260058_R06C02', '201662330194_R02C01', '201662330194_R01C01', '201662330194_R01C02']
gen_dosage = {'0/0':0,'0/1':1,'1/0':1,'1/1':2,'1/2':3,'2/2':4,'./.':5}
chromosomes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16','17', '18', '19', '20', '21', '22']
num_windows = 5

for chrom in chromosomes:
    # loop over chromosomes
    sc_merg_chr_wind_check = pd.read_csv(f'../server/outputs/windows/GM12878_SC_merged_chr{chrom}_window{window}_check.vcf.gz', header=0, sep='\t', dtype='object')
    # read in the single-cell merged chromosome VCF file
    for window in range(num_windows):
        # loop over windows
        current_window_bk = pd.read_csv(f'../server/outputs/windows/GM12878_gDNA_merged_chr{chrom}_window{window}_check.vcf', header=0, sep='\t')
        # read in the current bulk VCF window file
        
        print(sc_merg_chr_wind_check.shape[0], current_window_bk.shape[0])
        
        for sample in samples:
            # loop over samples
            dict_pre = []
            # create an empty list to hold the results for this sample and window
            with open(f'../server/results/windows/maskedpos_chr{chrom}_window{window}.txt','w') as output:
                output.write(str(current_window_bk.POS.tolist()))
            # write the list of positions in the bulk VCF window to a file
            # note: changed "bk_merg_chr" to "current_window_bk"
            
            vec_bulk = current_window_bk[sample].map(gen_dosage).tolist()
            vec_sing = sc_merg_chr_wind_check[sample].map(gen_dosage).tolist()
            # create lists of genotypes for the current sample in the bulk and single-cell data
            
            j_value = jaccard_score(vec_bulk, vec_sing, average='macro')
            # calculate the Jaccard similarity score for the current sample and window
            j_dict = {'sample': sample, 'chromosome': chrom, 'window': window, 'j_score': j_value}
            # create a dictionary with the sample, chromosome, window, and Jaccard score
            dict_pre.append(j_dict)
            # add the dictionary to the list for this sample and window
            
            pd.DataFrame(dict_pre).to_excel(f'../server/results/windows/jaccard_scores_{sample}_chr{chrom}_window{window}_pre.xlsx', index=False)
            # write the list of Jaccard scores for this sample and window to an Excel file
            # note: changed "pos_seed" to "current_window_bk.POS"


4581 4581
4581 4581
4581 4581
4581 4581
4581 4581
4792 4792
4792 4792
4792 4792
4792 4792
4792 4792
3527 3527
3527 3527
3527 3527
3527 3527
3527 3527
2877 2877
2877 2877
2877 2877
2877 2877
2877 2877
3541 3541
3541 3541
3541 3541
3541 3541
3541 3541
3284 3284
3284 3284
3284 3284
3284 3284
3284 3284
3110 3110
3110 3110
3110 3110
3110 3110
3110 3110
3139 3139
3139 3139
3139 3139
3139 3139
3139 3139
2415 2415
2415 2415
2415 2415
2415 2415
2415 2415
2727 2727
2727 2727
2727 2727
2727 2727
2727 2727
2852 2852
2852 2852
2852 2852
2852 2852
2852 2852
2904 2904
2904 2904
2904 2904
2904 2904
2904 2904
1786 1786
1786 1786
1786 1786
1786 1786
1786 1786
1910 1910
1910 1910
1910 1910
1910 1910
1910 1910
1999 1999
1999 1999
1999 1999
1999 1999
1999 1999
1828 1828
1828 1828
1828 1828
1828 1828
1828 1828
1914 1914
1914 1914
1914 1914
1914 1914
1914 1914
1511 1511
1511 1511
1511 1511
1511 1511
1511 1511
1368 1368
1368 1368
1368 1368
1368 1368
1368 1368
1510 1510
1510 1510
1510 1510
1510 1510
1510 1510


In [94]:

import glob

# create a list of all excel files matching the pattern
excel_files = glob.glob("../server/results/windows/jaccard_scores_*_pre.xlsx")

# concatenate all excel files into one dataframe
combined_df = pd.concat([pd.read_excel(file) for file in excel_files], ignore_index=True)

# write the concatenated dataframe to a new excel file
combined_df.to_excel("../server/results/windows/jaccard_scores_combined_pre.xlsx", index=False)

print(combined_df.shape[0]) # 22 x 5 x 5 = 550


In [100]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a larger figure
fig, ax = plt.subplots(figsize=(10, 6))

# Create the boxplot hue='perc'
sns.boxplot(data=combined_df, x='sample', y='j_score', hue='window', ax=ax, width=0.6)

# Add labels and title
ax.set_xlabel('Chromosome')
ax.set_ylabel('J Score')
ax.set_title('J Score Distribution by Chromosome and Percentage')

# Add legend outside the plot
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# Show the plot
plt.show()


TypeError: 'NoneType' object is not callable

<Figure size 1000x600 with 0 Axes>